In [0]:
library(dataiku)
library(rpart)

In [0]:
# Read the dataset as a R dataframe in memory
# Note: here, we only read the first 100K rows. Other sampling options are available
df_base_train <- dkuReadDataset("base_train", samplingMethod="head", nbRows=100000)

In [0]:
# Training structural equation for wind speed
# wind_speed = f(track_min_dist, eps)

base_wind_max_model <- rpart(wind_max ~ track_min_dist, 
                       data = df_base_train, 
                       method = "anova")


In [0]:
# Save the trained model in a Managed Folder
dkuManagedFolderPath <- dkuManagedFolderPath("base__folder")
saveRDS(model, file = paste0(dkuManagedFolderPath, "/trained_model.rds"))